# Benutech Data Joining

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

Fill in the dataset names saved from the querying notebook below

In [1]:
deed_name = ''
ols_name = ''
tax_name = ''

Select columns from deed, ols, tax that'll be used as features

In [2]:
deed_cols = ['COMPOSITE_PROPERTY_LINKAGE_KEY', 'COMPOSITE_TRANSACTION_ID','PROPERTY_INDICATOR_CODE',
                        'DEED_CATEGORY_TYPE', 'SALE_DATE', 'SALE_AMOUNT']
ols_cols = ['COMPOSITE_PROPERTY_LINKAGE_KEY','PURCHASE_AMOUNT','PURCHASE_COMBINED_LTV','ESTIMATED_EQUITY']
tax_cols = ['COMPOSITE_PROPERTY_LINKAGE_KEY', 'TOTAL_VALUE_CALCULATED', 'BEDROOMS', 'TOTAL_BATHROOMS_CALCULATED' ]


Some cleaning
1. Load dataframes with selected columns
2. Remove null dates in deed's sale dates, composite linkage keys, and property indicators. 
3. Sort deed by composite linkage keys and sale dates
4. Convert date column to datetime
5. Add a dummy column for censored properties. Censor = 0 -> property hasn't undergone a title change yet

In [3]:
deed = pd.read_csv(deed_name,usecols=deed_cols, low_memory=False)
deed = deed.loc[deed['SALE_DATE'] != '0000-00-00']
deed.sort_values(by = ['COMPOSITE_PROPERTY_LINKAGE_KEY', 'SALE_DATE'], inplace = True)
deed.dropna(subset=['COMPOSITE_PROPERTY_LINKAGE_KEY', 'PROPERTY_INDICATOR_CODE'], inplace=True)
deed["censor"] = 1
deed = deed[pd.to_datetime(deed['SALE_DATE'], errors='coerce').notna()]
deed['SALE_DATE'] = pd.to_datetime(deed['SALE_DATE'])

ols = pd.read_csv(ols_name,usecols=ols_cols, low_memory = False)
ols.dropna(subset=['COMPOSITE_PROPERTY_LINKAGE_KEY'], inplace=True)


tax = pd.read_csv(tax_name, usecols=tax_cols, low_memory = False)


Making labels for model and feature engineering

In [4]:
def get_targets(df):
    """a function that create censor (title change) and label(time that a property took to go through title changes)
    df: a deed df
    
    return two dictionaries, one that maps unique transaction ids to the time until the next title change or time until
    present if a title change hasn't occurred. The other dictionary contains the transaction ids mapping to an indicator
    of a title change or not ie the censor column.
    """
    
    # a dictionary of {property key: an array of timestamps of Sale Date
    deed_dates = df
    ids = {}
    dates = {}
    for idx, row in deed_dates.iterrows():
        key = row.COMPOSITE_PROPERTY_LINKAGE_KEY
        if key not in dates.keys():
            ids[key] = [row.COMPOSITE_TRANSACTION_ID]
            dates[key] = [row.SALE_DATE]
        else:
            ids[key].append( row.COMPOSITE_TRANSACTION_ID)
            dates[key].append(row.SALE_DATE )
    # a dictionary of {proporty key: an array of difference in Sale Date}
    diff_dict = {}
    c_ids = []
    for key in list(dates.keys()):
        date_list = dates[key] #all the sale dates of that property
        id_list = ids[key]
        if(len(date_list) > 1):
            for i in range(len(date_list)-1):
                current = date_list[i]
                next_d = date_list[i+1]
                diff = next_d - current
                diff_dict[id_list[i]] = diff
        last_d = pd.to_datetime("today") - date_list[-1]
        diff_dict[id_list[-1]] = last_d
        c_ids.append(id_list[-1])
    return diff_dict, c_ids


Use the helper function to get the labels, convert days to a float. Everything is done in deed since it contains unique transaction ids

In [5]:
diff_dict, c_ids = get_targets(deed)
deed['date_difference'] = deed['COMPOSITE_TRANSACTION_ID'].map(diff_dict)
deed['date_difference'] = deed['date_difference'] / pd.to_timedelta(1, unit='D')

Use the censor dictionary to create the censor column

In [6]:
deed.set_index('COMPOSITE_TRANSACTION_ID', inplace=True)
for i in c_ids:
  deed.at[i, 'censor'] = 0
deed.reset_index(inplace=True)

Joining
1. Left join all the dataframes on deed
2. Remove nulls in ols, select arms length transfers by looking at deed category type for grant and foreclosure deeds

In [7]:
deed_ols = deed.join(ols.set_index('COMPOSITE_PROPERTY_LINKAGE_KEY'), on = 'COMPOSITE_PROPERTY_LINKAGE_KEY', how='left')
full_df = deed_ols.join(tax.set_index('COMPOSITE_PROPERTY_LINKAGE_KEY'), on = 'COMPOSITE_PROPERTY_LINKAGE_KEY', how='left')
full_df.dropna(subset=['PURCHASE_AMOUNT', 'ESTIMATED_EQUITY'], inplace=True)
full_df = full_df.loc[full_df['DEED_CATEGORY_TYPE'].isin(['G', 'U'])]
full_df = full_df.loc[full_df.date_difference != 0]

Saving

In [8]:
full_df.to_csv('county_full_df.csv')